In [1]:
import argparse
import numpy as np
from sklearn.decomposition import PCA
#from matplotlib import pyplot as plt

In [2]:
#def make_rna_matrix(output_path):
    # make dictionary with biosample_matched_dnase and gene as keys, counts as value
output_path = '/srv/scratch/wychen66/get_correlation/rna_outputs/'
d = {}
biosample_matched_dnase_set=set()
eid_matched_dnase_set=set()
gene_id_set=set()
bios_cell_d={}
bios_eid_d={}
bios_color_d={}

with open('/mnt/data/integrative/metadata/rna_metadata_2016-12-05.txt','r') as f1: 
    for l1 in f1:
        if "rsem_quant_file" in l1: # skip header line
            continue
        if "UW_Stam" not in l1: # only keep data from Stam lab
            continue
        items = l1.strip().split('\t')   
        if items[7] == "":
            continue
        biosample_matched_dnase = items[7]
        rsem_quant_file = items[6]
        eid_matched_dnase = items[8]
        bios_cell_d[biosample_matched_dnase] = items[10]
        bios_color_d[biosample_matched_dnase] = items[12]
        
        bios_eid_d[biosample_matched_dnase] = eid_matched_dnase
        
        biosample_matched_dnase_set.add(biosample_matched_dnase)
        eid_matched_dnase_set.add(eid_matched_dnase)

        with open(rsem_quant_file,'r') as f2: # open rsem link
            for l2 in f2:
                if "count" in l2:
                    continue
                link = l2.split('\t')
                gene_id = link[0] # did not take transcript_id
                expected_count = link[4] 
                gene_id_set.add(gene_id)

                d[(gene_id, biosample_matched_dnase)] = expected_count

biosample_matched_dnase_list=list(biosample_matched_dnase_set)
eid_matched_dnase_list=list(eid_matched_dnase_set)
gene_id_list=list(gene_id_set)

np.savetxt(output_path+'rna_biosample_matched_dnase_list.txt', biosample_matched_dnase_list, fmt="%s")
np.savetxt(output_path+'rna_gene_id_list.txt', gene_id_list, fmt="%s")

In [3]:
### This cell contains code from correlate_rna_and_dnase.py
ids=[]
locs=[]
with open('/srv/scratch/wychen66/get_correlation/rna_outputs/all_genes_annotation.gtf', 'r') as f:
    for l in f:
        items = l.split(';')[0].split('\t')  
        ids.append(items[8])
        locs.append(items[0]+'\t'+items[3]+'\t'+items[4])

In [4]:
# make matrix where rows are gene_id, columns are eid_matched_dnase, and values are counts.
len_bio = len(biosample_matched_dnase_list)
len_gen = len(gene_id_list)
len_eid = len(eid_matched_dnase_list)

rna_matrix = np.empty([len_gen, len_bio], dtype=list)
rna_d={}

for gene_idx in range(len_gen):
    current_gene = gene_id_list[gene_idx]
    for bios_idx in range(len_bio):
        current_bios = biosample_matched_dnase_list[bios_idx]
        for match_idx in range(len_bio):
            current_match = biosample_matched_dnase_list[match_idx]
            current_counts = d.get((current_gene, current_bios))
            rna_matrix[gene_idx, bios_idx] = current_counts
            
            rna_d[(locs[gene_idx].replace("\t",","),current_bios)] = current_counts


rna_d_array = np.array(list(rna_d.items()))
print(rna_d_array.shape)

# save unlabeled rna matrix to txt
np.savetxt(output_path + 'rna_counts_gene-by-cellline_unlabeled.txt', rna_matrix,fmt="%s",delimiter='\t')

# this file is rna_d needed for correlate_rna_and_dnase.py
np.savetxt(output_path + 'rna_d_key_loc_sample_val_counts.txt', rna_d_array, fmt="%s",delimiter='\t')

print('rna matrix shape:', rna_matrix.shape)
print(len(rna_d))

(7110876, 2)
rna matrix shape: (57820, 123)
7110876


In [5]:
#print(len(locs[1]))
#print(locs[1])
#print(locs[1].replace("\t",","))

In [6]:
# get cell id code:
color_list = []
tissue_list = []
for i in np.arange(len(biosample_matched_dnase_list)):
    color = bios_color_d.get(biosample_matched_dnase_list[i])
    color_list.append(color)
    tissue = bios_cell_d.get(biosample_matched_dnase_list[i])
    tissue_list.append(tissue)
print(len(color_list), len(tissue_list))
print(len(biosample_matched_dnase_list))
print(biosample_matched_dnase_list[0:2])

123 123
123
['ENCSR174JMM_ENCSR392GCE.Fetal_Heart.UW_Stam.DNase-seq', 'ENCSR437KLY_ENCSR757EPJ.Fetal_Kidney.UW_Stam.DNase-seq']


In [7]:
# establish cell type for each pca data point
print('rna matrix shape:', rna_matrix.shape)

print(len(biosample_matched_dnase_list)) # this is the final rna matrix's column label
np.savetxt(output_path+'rna_matrix_column_labels.txt', biosample_matched_dnase_list, fmt="%s",delimiter='\t')

print('bios list starts:', biosample_matched_dnase_list[0:2])
print('number of blanks in bios list:', biosample_matched_dnase_list.count(''))

rna matrix shape: (57820, 123)
123
bios list starts: ['ENCSR174JMM_ENCSR392GCE.Fetal_Heart.UW_Stam.DNase-seq', 'ENCSR437KLY_ENCSR757EPJ.Fetal_Kidney.UW_Stam.DNase-seq']
number of blanks in bios list: 0


In [8]:
# add top row with eid_matched_danse of each col, and add left column with gene_id of each row
rna_matrix_labeled = np.insert(rna_matrix, 0, gene_id_list, axis=1) 
bios_matched_dnase_list_title = [''] + biosample_matched_dnase_list # first 2 items are empty
rna_matrix_labeled = np.insert(rna_matrix_labeled, 0, [bios_matched_dnase_list_title], axis=0) 

txt_name2 = output_path + 'rna_counts_gene-by-cellline_labeled.txt'
np.savetxt(txt_name2, rna_matrix_labeled,fmt="%s",delimiter='\t')
#awk '{ print $1 }' test_output 
print(bios_matched_dnase_list_title[0:3])

['', 'ENCSR174JMM_ENCSR392GCE.Fetal_Heart.UW_Stam.DNase-seq', 'ENCSR437KLY_ENCSR757EPJ.Fetal_Kidney.UW_Stam.DNase-seq']


In [9]:
# first sort rna matrix so that rows of gene_id are ordered according to chromosomal location
sorted_rna_matrix = np.zeros([rna_matrix.shape[0],rna_matrix.shape[1]])
print(sorted_rna_matrix.shape)

gene_labels=[]
with open('/srv/scratch/wychen66/get_correlation/rna_outputs/rna_gene_location_list_with_row_number_sorted.bedGraph', 'r') as f:
    row_count = 0
    for l in f:
        idx = int(l.strip().split('\t')[3])
        gene_labels.append(l.strip().split('\t')[0:3])
        sorted_rna_matrix[row_count,:] = rna_matrix[idx,:]
        row_count += 1

print(len(gene_labels))
np.savetxt(output_path+'rna_counts_gene-by-cellline_sorted_unlabeled.txt', sorted_rna_matrix,fmt="%s",delimiter='\t')
np.savetxt(output_path+'rna_counts_gene-by-cellline_sorted_unlabeled_row_label.txt', gene_labels,fmt="%s",delimiter='\t')
np.savetxt(output_path + 'rna_counts_gene-by-cellline_sorted_column_label.txt', biosample_matched_dnase_list,fmt="%s",delimiter='\t')

(57820, 123)
57820


In [10]:
'''
# below prepares for pca analysis
rna_transpose = np.transpose(sorted_rna_matrix)
print('orignal dimensions:', rna_transpose.shape)
print(rna_transpose[0:2,0:2])

# normalize over sequencing depth
col_sum = np.sum(rna_transpose,axis=1)
col_sum = col_sum.reshape(col_sum.shape[0],-1) # from (122,) to (122,1)
#rna_transpose_depth_normalized = 10000000* np.divide(rna_transpose,col_sum)
rna_transpose_depth_normalized = np.arcsinh(10000000* np.divide(rna_transpose,col_sum))
print(rna_transpose_depth_normalized[0:2,0:2])

# plot PCA of rna matrix
pca = PCA(n_components=2)
pca.fit(rna_transpose_depth_normalized) # fit model to data
rna_pca = pca.transform(rna_transpose_depth_normalized) # reduce dimensions
print('reduced dimensions:', rna_pca.shape)

# plot PCA output (2)
cell_n = np.arange(len(biosample_matched_dnase_list))
print('pca variance ratio:', pca.explained_variance_ratio_)

plt.figure(figsize=(10, 10))

for i, c, cell_label in zip(cell_n, color_list, tissue_list): 
    plt.scatter(rna_pca[i, 0], rna_pca[i, 1], color=c, label=cell_label)
    
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('rna PCA')
plt.show()

# https://matplotlib.org/examples/statistics/violinplot_demo.html
pos = np.arange(sorted_rna_matrix.shape[1]) + 1
data = sorted_rna_matrix
plt.plot(nrows=5, ncols=5,figsize=(10,10))
plt.violinplot(data, pos, points=20, widths=1, showmeans=True, showextrema=True, showmedians=True)

plt.title('rna violin plot')
plt.show()

# density plot across ~ 100 samples overlaid on eachother

print(sorted_rna_matrix.shape)
x = np.arange(sorted_rna_matrix.shape[0])

y = []
for i in np.arange(sorted_rna_matrix.shape[1]):
    y.append(sorted_rna_matrix[:,i])

y = np.transpose(y)

plt.figure(figsize=(10, 10))
plt.plot(x,y,'k',linewidth=0.1)
plt.title('rna density plot across 123 samples')
plt.xlabel('57820 genes')
plt.ylabel('counts')
plt.show()


# make frequency plot of each column of dnase_matrix taking on various values
print('sorted_rna_matrix shape:',sorted_rna_matrix.shape)
print('matrix maximum:',np.amax(sorted_rna_matrix), '  matrix minimum:',np.amin(sorted_rna_matrix))

freq_matrix = np.zeros([123,100])
tick_marks = np.linspace(-1, 22889435, 101)

for sample in np.arange(123):
    left_tick = 0
    right_tick = 1
    for frame in np.arange(100):
        freq_matrix[sample,frame] = ((left_tick < sorted_rna_matrix[:,sample]) & (sorted_rna_matrix[:,sample] < right_tick)).sum()
        left_tick += 1
        right_tick += 1

#print(freq_matrix)
#print(np.sum(freq_matrix, axis=0))

x = tick_marks[0:100]
y = np.transpose(freq_matrix)

plt.figure(figsize=(10, 10))
plt.plot(x,y,'k',linewidth=0.1)
plt.title('frequency plot of each column of sorted_rna_matrix taking on various values across 123 samples')
plt.xlabel('count values in sorted_rna_matrix')
plt.ylabel('frequency of count value')
plt.show()
'''

"\n# below prepares for pca analysis\nrna_transpose = np.transpose(sorted_rna_matrix)\nprint('orignal dimensions:', rna_transpose.shape)\nprint(rna_transpose[0:2,0:2])\n\n# normalize over sequencing depth\ncol_sum = np.sum(rna_transpose,axis=1)\ncol_sum = col_sum.reshape(col_sum.shape[0],-1) # from (122,) to (122,1)\n#rna_transpose_depth_normalized = 10000000* np.divide(rna_transpose,col_sum)\nrna_transpose_depth_normalized = np.arcsinh(10000000* np.divide(rna_transpose,col_sum))\nprint(rna_transpose_depth_normalized[0:2,0:2])\n\n# plot PCA of rna matrix\npca = PCA(n_components=2)\npca.fit(rna_transpose_depth_normalized) # fit model to data\nrna_pca = pca.transform(rna_transpose_depth_normalized) # reduce dimensions\nprint('reduced dimensions:', rna_pca.shape)\n\n# plot PCA output (2)\ncell_n = np.arange(len(biosample_matched_dnase_list))\nprint('pca variance ratio:', pca.explained_variance_ratio_)\n\nplt.figure(figsize=(10, 10))\n\nfor i, c, cell_label in zip(cell_n, color_list, tissu

In [11]:
'''
def Main():
    parser = argparse.ArgumentParser()
    parser.add_argument('output_path', help='The desired path to the output files, ending with a forward slash.', type=str)
    args = parser.parse_args()
    result = make_rna_matrix(args.output_path)
    print('Path to the output: ' + str(args.output_path) + 'rna_counts_gene-by-cellline.txt')
    
if __name__ == '__main__':
    Main()
    '''

"\ndef Main():\n    parser = argparse.ArgumentParser()\n    parser.add_argument('output_path', help='The desired path to the output files, ending with a forward slash.', type=str)\n    args = parser.parse_args()\n    result = make_rna_matrix(args.output_path)\n    print('Path to the output: ' + str(args.output_path) + 'rna_counts_gene-by-cellline.txt')\n    \nif __name__ == '__main__':\n    Main()\n    "

In [12]:
# below cells are from make_dnase_matrix.py

In [13]:
dnase_matrix = np.loadtxt('/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/final_dnase_matrix.txt',dtype=float)

# bios_eid_for_dnase_matrix is the column labels for final_dnase_matrix.txt
dnase_bios_labels=[]
with open('/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/bios_eid_for_final_dnase_matrix.txt','r') as f1: 
    for l1 in f1:
        items = l1.strip().split('\t')
        dnase_bios_labels.append(items[0])

# dnase_labels is sample (also folder name) in dna_d:
dnase_labels =[] # this is reliable
with open('/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/folder_name.txt', encoding = 'ISO-8859-1') as f2: 
    for l2 in f2:
        l2=l2.strip()
        dnase_labels.append(l2)

# /srv/scratch/wychen66/get_correlation/dnase_outputs/idr_files_merged.bed is a list of dnase locations in matrix
dna_loc = []
with open('/srv/scratch/wychen66/get_correlation/dnase_outputs/idr_files_merged.bed','r') as f3: # a list with 762379 items
    for l3 in f3:
        l3 = l3.strip().split('\t')
        l3 = l3[0]+','+l3[1]+','+l3[2]
        dna_loc.append(l3)

# dnase_matrix has counts for dna_d, so make dna_d:
dna_d = {}
for r in np.arange(dnase_matrix.shape[0]):
    for c in np.arange(dnase_matrix.shape[1]):
        dna_d[(dna_loc[r],dnase_labels[c].replace(" ","_"))] = dnase_matrix[r,c] # is this right???
        

In [14]:
# code below are from correlate_rna_and_dnase.py

In [15]:
#http://www.gencodegenes.org/gencodeformat.html
all_genes_ann = '/srv/scratch/wychen66/get_correlation/rna_outputs/all_genes_annotation.gtf'
rna_gene_id = '/srv/scratch/wychen66/get_correlation/rna_outputs/rna_gene_id_list.txt'
rna_gene_loc = '/srv/scratch/wychen66/get_correlation/rna_outputs/'

rna_matrix = '/srv/scratch/wychen66/get_correlation/rna_outputs/rna_counts_gene-by-cellline_unlabeled.txt' # (57820, 123)
dnase_matrix = '/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/final_dnase_matrix.txt' # (762379,122)
rna_cell_labels = '/srv/scratch/wychen66/get_correlation/rna_outputs/rna_biosample_matched_dnase_list.txt' # 123
dnase_cell_labels = '/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/bios_eid_for_final_dnase_matrix.txt' #122
dnase_folder_name = '/srv/scratch/wychen66/get_correlation/dnase_outputs/second_try/folder_name.txt' # 122

rna_labels =[]
with open(rna_cell_labels,encoding = 'ISO-8859-1') as f1: 
    for l1 in f1:
        l1=l1.strip()
        rna_labels.append(l1)

rna_labels_set = set(rna_labels)
print('rna tissue labels:',len(rna_labels))

dnase_labels =[]
with open(dnase_folder_name, encoding = 'ISO-8859-1') as f2: 
    for l2 in f2:
        l2=l2.strip()
        dnase_labels.append(l2)

dnase_labels_set = set(dnase_labels)
print('dnase tissue labels:', len(dnase_labels))

# dictionary where key is index of rna cell list, and value is index of dnase cell list
d={}
for d_i in np.arange(len(dnase_labels)):
    r_i = rna_labels.index(dnase_labels[d_i])
    d[r_i] = d_i
    
print(len(d))

# convert gene_id (ids) in rna_matrix to chrom locations (locs)
genes =[]
with open(rna_gene_id, encoding = 'ISO-8859-1') as f3: 
    for l3 in f3:
        l3=l3.strip()
        genes.append(l3)

ids=[]
locs=[]
with open(all_genes_ann, 'r') as f:
    for l in f:
        items = l.split(';')[0].split('\t')  
        ids.append(items[8])
        locs.append(items[0]+'\t'+items[3]+'\t'+items[4])

# takes ~15 min to run
loc_list = []
counter = 0
for i in np.arange(len(genes)):
    idx = [ids.index(j) for j in ids if genes[i] in j][0] # [0] changes list to just int
    loc_list.append(locs[idx]+'\t'+str(counter))
    counter += 1

np.savetxt('/srv/scratch/wychen66/get_correlation/rna_outputs/rna_gene_location_list_with_row_number.bedGraph', loc_list, fmt="%s")

# loc of dnase peaks for dnase matrix is idr_files_merged.bed

rna tissue labels: 123
dnase tissue labels: 122
122


In [16]:
loc_d = {} # dictionary where key is rna loc and value is list of dnase locs < 1 MB
with open('/srv/scratch/wychen66/get_correlation/bed_overlap/dnase_within_1MB_of_rna.bed','r') as f4: # 32810450 lines
    for l4 in f4:
        items = l4.strip().split('\t')
        if tuple(items[0:3]) in loc_d:
            loc_d[tuple(items[0:3])].append(items[3:6]) # try to insert |t between chr, start & finish to make a list of char     
        else:
            loc_d[tuple(items[0:3])] = [items[3:6]]

In [17]:
print(tuple(items[0:3]))

('chr1', '206506678', '206506789')


In [18]:
# everything above runs fine, but need to comment out/re-code below:

In [19]:
#print(len(locs[1]))
#print(locs[1])
#print(locs[1].replace("\t",","))

To get rna-dnase correlation make 2 lists of sample len():
- list of rna-dna location pair
- list of rna-dna count correlation score across all the samples:
    - get rna count as a 122-long vector
    - get dna count as a 122-long vector
    - apply pearsonr / spearmanr to the vectors to get 1 correlation score

In [ ]:
# get loc list
loc_list = []

for rna_loc in loc_d: # loc_d = {} is a dictionary where key is rna loc and value is list of dnase locs < 1 MB
    dna_loc_list = loc_d[rna_loc]
    for dna_loc in dna_loc_list:
        loc_list.append((rna_loc, dna_loc)) # stores tuple of rna-dnase loc pairing
     
print('length of loc_list (rna-dnase location pairings): ', len(loc_list))

length of loc_list (rna-dnase location pairings):  32810450


In [ ]:
# get correlation scores for each rna-dnase pair in loc_list
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
score_list_pearson = []
score_list_spearman = []

# make 1 pass through this and give both real and shuffled corr

counter=0
for rna_loc in loc_d: # for each rna loc
    dna_loc_list = loc_d[rna_loc] # get list of dna loc <1MB
    
    rna_ct_all_samples = [] # list of rna counts across 122 samples for 1 specific rna loc
    for rna_idx in d.keys(): # for idx to each sample
        sample = rna_labels[rna_idx] ### eval for chr21
        rna_loc_joint = rna_loc[0]+","+rna_loc[1]+","+rna_loc[2] # chr10,13771383,13798200
        rna_ct_all_samples.append(rna_d[(rna_loc_joint, sample)]) # got rna counts vector for 1 rna loc
    
    for dna_loc in dna_loc_list: # for each <1MB dna loc
        
        dna_ct_all_samples = [] # list of dna counts across 122 samples for 1 specific dnase loc
        dna_loc_joint = dna_loc[0]+","+dna_loc[1]+","+dna_loc[2] # see if necessary
        for rna_idx in d.keys(): # for idx to each sample
            sample = rna_labels[rna_idx] 
            dna_ct_all_samples.append(dna_d[(dna_loc_joint, sample)]) # got dnase counts vector for 1 <1MB dnase
        
        rna_vec = np.asarray(rna_ct_all_samples).astype(np.float)
        dna_vec = np.asarray(dna_ct_all_samples).astype(np.float)
    
        score_list_pearson.append(pearsonr(rna_vec, dna_vec))
        score_list_spearman.append(spearmanr(rna_vec, dna_vec))
                       
    counter += 1
    if (counter%10) == 100000: 
        print(counter)

print(len(score_list_pearson), len(score_list_spearman), len(rna_ct_all_samples), len(dna_ct_all_samples))

/home/wychen66/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/home/wychen66/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3003: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/wychen66/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3004: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/wychen66/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/wychen66/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/wychen66/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in 

In [ ]:
# save loc_list, score_list_pearson, score_list_spearman to txt
np.savetxt('/srv/scratch/wychen66/get_correlation/list_of_rna-dnase-pairs_for_correlation.txt', loc_list, fmt="%s")
np.savetxt('/srv/scratch/wychen66/get_correlation/pearson_correlation_for_list_of_rna-dnase-pairs.txt', score_list_pearson, fmt="%s")
np.savetxt('/srv/scratch/wychen66/get_correlation/spearman_correlation_for_list_of_rna-dnase-pairs.txt', score_list_spearman, fmt="%s")

In [ ]:
# this cell crashes the server:
# make dictionary (counts_all_d) where key is cell type, val is counts_d, containing dnase counts for each rna counts
'''
counts_all_d = {} 

for rna_idx in d.keys(): # for each cell type (rna_idx is index of rna cell label list)
    sample = rna_labels[rna_idx]
    counts_d = {} # dictionary where key is rna counts and value is total counts of dnase < 1MB for 1 cell type
    
    for rna_loc in loc_d: # change loc_d code above so that rna_loc can match rna_d keys in format??
        rna_loc_joined = rna_loc[0]+","+rna_loc[1]+","+rna_loc[2] # chr10,13771383,13798200
        rna_count = rna_d[(rna_loc_joined, sample)]
        dna_loc_list = loc_d[rna_loc]
        
        dna_count = 0
        for dna_loc in dna_loc_list: # change dna_loc_list code above so that dna_loc can match dna_d keys in format??
            dna_loc = dna_loc[0]+","+dna_loc[1]+","+dna_loc[2]
            dna_count += dna_d[(dna_loc, sample)] # keyError
        counts_d[rna_count] = dna_count # should have 57695 items
    counts_all_d[rna_idx] = counts_d 

print(len(counts_d))
'''

notes: 
- the biosample_matched_rna will establish rna-dnase pairs for later correlation
- but the eid will be the cell line